In [612]:
from glob import glob
import csv
import os
import re
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords


dir = os.chdir('/Users/jethin/Google_Drive/Capstone/MSDS_Capstone-master/data/csv_daily/Bitcoin')

In [613]:
file_name="20180420"

In [614]:
import pandas as pd

df = pd.read_csv(file_name +".csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [615]:
 df["tweet1"] =df.text.str.replace('RT','')

In [616]:
 df.tweet2 = df.tweet1.str.replace('@[^\s]+','')

In [617]:
 df.tweet3 = df.tweet2.str.replace(r'#([^\s]+)',r'\1')

In [618]:
 df.tweet4 = df.tweet3.str.replace('https?://[^\s]+','')

In [619]:
df.tweet4.head()

0      Private cryptoinvestors are receiving profit...
1    Military robots are already being used in conf...
2      You can superimpose the graphs for\n- male s...
3      Get Awesome Lighting in Your Home Now! \n \n...
4      Illinois woman suspected of moving through s...
Name: tweet1, dtype: object

In [620]:
df.tweet5=df.tweet4.str.replace('[$?]+','')

In [621]:
df["tweet"]=df.tweet5

In [622]:
df["user_followers_count"]=pd.to_numeric(df.user_followers_count, errors='ignore')

In [623]:
df["user_followers_count"]=df.user_followers_count.fillna(0)

In [624]:
  df_filtered = df

In [625]:
df_filtered["tweet"]=df_filtered.tweet.str.replace('\r','')

In [626]:
df_filtered["tweet"]=df_filtered.tweet.str.replace('\n','')

In [627]:
df_filtered["tweet"]=df_filtered.tweet.str.replace('"', '')

In [628]:
df_filtered = df[df.tweet.str.len()>30]

In [629]:
df_filtered["user_followers_count"]= df_filtered.user_followers_count.astype(int)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [630]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

In [631]:
def print_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(snt)))

In [632]:
from textblob import TextBlob

In [633]:
df['tweet'] = df['tweet'].astype(str)

In [634]:
df.shape[0]

191811

In [635]:
for i in range(0,df.shape[0]-1):
    df.ix[i,"text_sentiment_score"] = TextBlob(df.ix[i,'tweet']).sentiment.polarity
    df.ix[i,"subjective"] = TextBlob(df.ix[i,'tweet']).sentiment.subjectivity
    df.ix[i,"vader_pos"]=analyser.polarity_scores((df.ix[i,'tweet']))["pos"]
    df.ix[i,"vader_neg"]=analyser.polarity_scores((df.ix[i,'tweet']))["neg"]
    df.ix[i,"vader_neu"]=analyser.polarity_scores((df.ix[i,'tweet']))["neu"]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [636]:


def vader_sen(data):
    if data['vader_neu'] == 1:
        return 'Neutral'
    elif data['vader_pos'] >data['vader_neg']:
        return 'Positive' 
    else:
        return 'Negative'
    

def vader_score(data):
    if data['vader_neu'] == 1:
        return data['vader_neu']
    elif data['vader_pos'] >data['vader_neg']:
        return data['vader_pos'] 
    else:
        return data['vader_neg']
    



In [637]:
df['vader_sentiment'] = df.apply(vader_sen, axis=1)
df['vader_score'] = df.apply(vader_score, axis=1)

In [638]:
df=df[df.tweet.str.len()>30]

In [639]:
df=df[df.user_followers_count.str.len()<12]

In [640]:
df.shape

(185660, 24)

In [641]:
dir = os.chdir('/Users/jethin/Google_Drive/Capstone/MSDS_Capstone-master/data/csv_daily/Bitcoin/daily_sentiment')

In [642]:
df.to_csv(file_name+".csv")